In [ ]:
!pip install opencv-python

Keyring is skipped due to an exception: 'keyring.backends'
Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Would remove:
    /opt/anaconda3/lib/python3.7/site-packages/cv2/*
    /opt/anaconda3/lib/python3.7/site-packages/opencv_python-4.11.0.86.dist-info/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
/bin/sh: y: command not found
Keyring is skipped due to an exception: 'keyring.backends'
/bin/sh: y: command not found


In [2]:
#import opencv
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Load the image
image = cv2.imread('maze1.png')

#convert the image to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  

#display the image
plt.imshow(image_rgb)
plt.show()

error: OpenCV(4.11.0) /private/var/folders/qk/535hwtln5xl9jpx91yqhrz940000gn/T/pip-install-lc90mebo/opencv-python_944df4d1dfc24cf4b329fb4ab8a3d2ac/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [8]:
# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Threshold the image to get binary image (black and white)
_, binary = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

# Define color detection functions
def create_color_mask(image, lower_bound, upper_bound):
    """Create a mask for
     a specific color range in an image."""
    return cv2.inRange(image, lower_bound, upper_bound)

def find_centroid(mask):
    """Find the centroid of a binary mask."""
    M = cv2.moments(mask)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        return (cX, cY)
    return None

def detect_start_end_points(image):
    """Detect start (green) and end (red) points in the image."""
    # Define color thresholds
    lower_green = np.array([0, 1, 0])
    upper_green = np.array([100, 255, 100])
    lower_red = np.array([1, 0, 0])
    upper_red = np.array([255, 100, 255])
    
    # Create masks for green and red
    green_mask = create_color_mask(image, lower_green, upper_green)
    red_mask = create_color_mask(image, lower_red, upper_red)
    
    # Find centroids
    start_point = find_centroid(green_mask)
    end_point = find_centroid(red_mask)
    
    print(f"Start point (green): {start_point}")
    print(f"End point (red): {end_point}")
    
    return start_point, end_point

def create_navigation_grid(binary_image):
    """Create a grid for pathfinding where 1 is traversable and 0 is obstacle."""
    grid = np.ones_like(binary_image)
    grid[binary_image == 0] = 0  # Mark black areas as obstacles
    return grid

def highlight_points_on_image(image, points, color):
    """Highlight specific points on an image with a given color."""
    highlighted_image = image.copy()
    for point in points:
        if point:
            highlighted_image[point[1], point[0]] = color
    return highlighted_image

def visualize_grid(grid, start_point, end_point):
    """Visualize the pathfinding grid with start and end points."""
    plt.figure(figsize=(10, 10))
    plt.imshow(grid, cmap='binary')
    plt.title('Pathfinding Grid (White: Traversable, Black: Obstacles)')
    
    # Mark start and end points
    if start_point:
        plt.plot(start_point[0], start_point[1], 'go', markersize=10)
    if end_point:
        plt.plot(end_point[0], end_point[1], 'ro', markersize=10)
    
    plt.grid(True, linestyle='-', alpha=0.7)
    plt.show()

def heuristic(a, b):
    """Calculate the Euclidean distance between two points."""
    return np.sqrt((b[0] - a[0]) ** 2 + (b[1] - a[1]) ** 2)

def a_star(grid, start, end):
    """Implement A* pathfinding algorithm."""
    import heapq
    
    # Check if start or end points are None
    if start is None or end is None:
        print("Start or end point is missing")
        return None
    
    # Convert start and end to (y, x) format for grid indexing
    start = (start[1], start[0])  # Convert from (x, y) to (y, x)
    end = (end[1], end[0])        # Convert from (x, y) to (y, x)
    
    neighbors = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
    close_set = set()
    came_from = {}
    gscore = {start: 0}
    fscore = {start: heuristic(start, end)}
    oheap = []
    
    heapq.heappush(oheap, (fscore[start], start))
    
    while oheap:
        current = heapq.heappop(oheap)[1]
        
        if current == end:
            return reconstruct_path(came_from, start, current)
        
        close_set.add(current)
        
        for i, j in neighbors:
            neighbor = current[0] + i, current[1] + j
            
            if not is_valid_position(grid, neighbor):
                continue
                
            tentative_g_score = gscore.get(current, float("inf")) + heuristic(current, neighbor)
            
            if neighbor in close_set and tentative_g_score >= gscore.get(neighbor, float("inf")):
                continue
            
            if tentative_g_score < gscore.get(neighbor, float("inf")) or neighbor not in [i[1] for i in oheap]:
                came_from[neighbor] = current
                gscore[neighbor] = tentative_g_score
                fscore[neighbor] = tentative_g_score + heuristic(neighbor, end)
                heapq.heappush(oheap, (fscore[neighbor], neighbor))
    
    return None

def is_valid_position(grid, position):
    """Check if a position is valid and traversable in the grid."""
    y, x = position
    if 0 <= y < grid.shape[0] and 0 <= x < grid.shape[1]:
        return grid[y][x] == 1
    return False

def reconstruct_path(came_from, start, end):
    """Reconstruct the path from start to end using the came_from dictionary."""
    path = []
    current = end
    while current in came_from:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()
    # Convert path back to (x, y) format
    return [(y, x) for x, y in path]

def visualize_path(image, path):
    """Visualize the path on the image."""
    if not path:
        print("No path found")
        return
    
    # Create a copy of the original image to draw the path
    result_image = image.copy()
    
    # Draw the path in yellow
    for x, y in path:
        result_image[y, x] = [255, 255, 0]  # Yellow color
    
    # Display the result
    plt.figure(figsize=(10, 10))
    plt.imshow(result_image)
    plt.title("Path from Green to Red")
    plt.show()
    
    print(f"Path found with {len(path)} points")

# Main execution flow
start_point, end_point = detect_start_end_points(image)
grid = create_navigation_grid(binary)
visualize_grid(grid, start_point, end_point)
path = a_star(grid, start_point, end_point)

visualize_path(image_rgb, path)

# Store the path coordinates
path_coordinates = path if path else []


error: OpenCV(4.11.0) /private/var/folders/qk/535hwtln5xl9jpx91yqhrz940000gn/T/pip-install-lc90mebo/opencv-python_944df4d1dfc24cf4b329fb4ab8a3d2ac/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
